In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl
%gui qt

import hyp
import eeg
import utils
import os
import json
import pandas as pd

In [ ]:
settings = json.load(open('./settings.json'))

path = settings['general']['dataset_dir']
subs = '24'

if (subs == 'all') or (subs == ['all']):
    subs = []

    for file in os.listdir(path):
        if file.startswith('sub-'):
            _, file = file.split('sub-')
            subs.append(file)

    subs.sort()

if type(subs) is str: subs = [subs]

for sub in subs:
    settings['general']['sub'] = sub
    settings['general']['ses'] = '40w'
    settings['general']['run'] = 'all'

    os.remove('./settings.json')
    with open('./settings.json', 'w') as f:
        json.dump(settings, f)

    sub = settings['general']['sub']
    ses = settings['general']['ses']

    path = os.path.join(settings['general']['dataset_dir'], settings['general']['eeg_subdir'].split('ses')[0].replace(
            '<SUB>', sub))

    ses = utils.check_all_setting(ses, 'ses', 'w', path)

    for _ses in ses:
        run = settings['general']['run']

        output_dir = utils.create_output_dir(settings['general']['output_dir'], sub, _ses)

        path = os.path.join(settings['general']['dataset_dir'], settings['general']['eeg_subdir'].replace(
            '<SUB>', sub).replace('<SES>', _ses))
        
        run = utils.check_all_setting(run, 'run', settings['hyp']['suffix'], path)

        dfs = []

        for _run in run:
            filename = utils.get_filename(settings, _ses, _run, settings['hyp']['suffix'])
            filename = os.path.join(path, filename)

            dfs.append(hyp.analysis.get_hyp_df(filename, settings))

        df = pd.concat(dfs, ignore_index=True)
        df = df[['label', 'date']]

        print(df)

        df = hyp.analysis.normalize_dataframe(df, tolerance=30)

        print(df)

        hours = hyp.analysis.count_full_hours(df)
        print(hours)

